![lightning](./lightning.png)

**그냥 싹다 class 내부 def 함수로 정의하고 마지막에 모델 객체 생성 후 fit하면 모든걸 다 해결해주는 느낌**

- 기존 pytorch는 DataLoader, Model, optimizer, Training roof 등을 전부 따로따로 코드 구현
- Lightning : Model class 안에 이 모든 것을 한 번에 구현
    - Lightning Module : 모델 내부 구조 설계
        - 구조, 전처리, 손실함수 등 정의
    - Trainer : 모델 학습 관련 (위 그림에서 파란색 부분!)
        - epoch, batch, 저장, 로그 관련 등
        - 코드로는 pl.Trainer()로 하면 끝 : 특히 두줄로 많이 줄었음

In [3]:
import torch
import pytorch_lightning as pl
from torch import Tensor, nn
from sklearn.datasets import load_boston
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F
import numpy as np

### Lightning Module은 6가지로 구성됨

In [4]:
class Classifier(pl.LightningModule):
    def __init__(self): # 초기화 메서드
        super().__init__()
        self.model = nn.Sequential(
            ...
        )

    def forward(self, x):
        pass # 모델의 추론결과 : f(x)식에 입력된 x로부터 예측된 y를 얻는 것을 forward 연산이라 함

    def training_step(self, batch, batch_idx):
        pass # 단일 배치에서의 loss 반환 <- train loop으로 자동 반복

    def validation_step(self, batch, batch_idx):
        pass

    def test_step(self, batch, batch_idx):
        pass

    def configure_optimizers(self):
        pass # 최적 파라미터를 찾을 때 사용할 optimizer, schduler 구현

### 'Boston House' example

In [7]:
class SklearnDataset(Dataset):
    def __init__(self, X: np.ndarray, y: np.ndarray):
        super().__init__()
        scaler = MinMaxScaler() 

        scaler.fit(X) 
        self.X = scaler.transform(X)
        self.Y = y

    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        x = self.X[idx].astype(np.float32)
        y = self.Y[idx].astype(np.float32)
        return x, y

In [8]:
X, y = load_boston(return_X_y=True)
bostonds = SklearnDataset(X, y)
train_loader = DataLoader(bostonds, batch_size=32, shuffle=True, drop_last=True, )

In [9]:
class LinRegModel(pl.LightningModule):
    def __init__(self, input_dim: int):
        super().__init__()
        self.linear = nn.Linear(in_features=13, out_features=1, bias=True)

    def forward(self, x):
        y_hat = self.linear(x)
        return y_hat

    def training_step(self, batch, batch_idx):
        x, y = batch
        # flatten any input
        x = x.view(x.size(0), -1)
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y, reduction="sum")
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-4)

In [10]:
trainer = pl.Trainer()
model = LinRegModel(input_dim=13)
trainer.fit(model, train_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\SeoJeongBin\miniconda3\envs\kotorch\lib\site-packages\pytorch_lightning\loops\utilities.py:89: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
Missing logger folder: c:\Users\SeoJeongBin\Desktop\작업\KoGPT2_ChatBot\Pytorch_lightning\lightning_logs

  | Name   | Type   | Params
----------------------------------
0 | linear | Linear | 14    
----------------------------------
14        Trainable params
0         Non-trainable params
14        Total params
0.000     Total estimated model params size (MB)
c:\Users\SeoJeongBin\miniconda3\envs\kotorch\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:236: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider 

Epoch 0:  20%|██        | 3/15 [00:00<00:00, 27.52it/s, loss=5.58e+05, v_num=0]

C:\Users\SeoJeongBin\AppData\Local\Temp\ipykernel_2596\1653378779.py:15: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(y_hat, y, reduction="sum")


Epoch 999: 100%|██████████| 15/15 [00:00<00:00, 96.22it/s, loss=2.84e+05, v_num=0] 

`Trainer.fit` stopped: `max_epochs=1000` reached.


Epoch 999: 100%|██████████| 15/15 [00:00<00:00, 87.26it/s, loss=2.84e+05, v_num=0]
